In [27]:
import pandas as pd
import numpy as np
import os

In [28]:
def extract_csv_data(csv_file, enumerate_num):
    df = pd.read_csv(f'inputs/{csv_file}')
    payments = df.shape[0] + 1
    payout_date = df['payouts.arrival_date'][0]
    payout_date = pd.to_datetime(payout_date)
    next_payout_sum = df['net_amount'].sum()

    output_str = f"""
    ID: {enumerate_num+1}
    Number of Payments: {payments}
    Date of Payout: {payout_date.strftime('%d-%b-%Y')}
    Payout Amount: £{round(next_payout_sum,2)}
    """

    print(output_str)
    return df, payout_date

In [29]:
def extract_payment_amounts(df, split = 'schedule'):
    sections = ['Squirrels', 'Beavers', 'Cubs', 'Scouts']
    payment_schedules = ['Activities', 'Monthly Subscriptions']

    result_subsets = []  # List to store subsets

    df = df.groupby(['resources.description'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()

    for section in sections:
        subset_section = df[df['resources.description'].str.contains(section, case=False)].copy()

        for schedule in payment_schedules:
            subset_schedule = subset_section[subset_section['resources.description'].str.contains(schedule, case=False)].copy()
            subset_schedule['Section'] = section
            subset_schedule['Payment Schedule'] = schedule
            subset_schedule['Section Payment Schedule'] = section + ": " + schedule

            result_subsets.append(subset_schedule)

    # Concatenate all subsets into a new DataFrame
    df = pd.concat(result_subsets, ignore_index=True)
    
    # Group so all sections payments for each schedule are together
    if split == 'schedule':
        df = df.groupby(['Section', 'Payment Schedule'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()
        df = df.sort_values(by=['Section', 'Payment Schedule'])
    elif split == 'section':
        df = df.groupby(['Section'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()
        df = df.sort_values(by=['Section'])
  
    return df

In [30]:
def sort_and_order_df(df):
    
   df['total_fees'] = df['gross_amount'] - df['net_amount']

   column_order = ['Section', 'Payment Schedule', 'net_amount', 'total_fees', 'gross_amount', 'gocardless_fees','app_fees']
    
   df = df[column_order]
  
   column_sums = df.sum()
   
   total_row = pd.DataFrame([column_sums.values], columns=column_sums.index, index=['Total'])
    
   df = pd.concat([df, total_row])

   currency_columns = ['net_amount', 'total_fees', 'gross_amount', 'gocardless_fees', 'app_fees']
   df[currency_columns] = df[currency_columns].round(2)
   # df = df.round(decimals=0)

   return df

In [31]:
def style_df(df):
    
    df.at['Total', 'Section'] = ''
    df.at['Total', 'Payment Schedule'] = ''
    
    # df = df.style.apply(lambda x: ['background-color: #008080']*len(x), subset=pd.IndexSlice[df.index[-1], :])
    # df = df.set_properties(**{'text-align': 'left'}, subset=[('Section')])
    # df = df.set_properties(**{'text-align': 'left'}, subset=[('Payment Schedule')])

    return df

In [32]:
def by_schedule():
    csv_files = [f for f in os.listdir('inputs') if f.endswith(".csv")]
    for idx, csv_file in enumerate(csv_files):
        df, payout_date = extract_csv_data(csv_file=csv_file, enumerate_num=idx)
        df = extract_payment_amounts(df)
        df = sort_and_order_df(df)
        df = style_df(df)
        display(df)
        df.to_excel(f'outputs/output_{payout_date}.xlsx', index=False)
by_schedule()
        


    ID: 1
    Number of Payments: 39
    Date of Payout: 12-Jan-2024
    Payout Amount: £489.68
    


,Section,Payment Schedule,net_amount,total_fees,gross_amount,gocardless_fees,app_fees
0,Beavers,Activities,43.20,1.80,45.0,0.93,0.87
1,Beavers,Monthly Subscriptions,34.44,1.56,36.0,0.87,0.69
2,Cubs,Monthly Subscriptions,344.40,15.60,360.0,8.70,6.90
3,Scouts,Activities,56.16,1.84,58.0,0.71,1.13
4,Squirrels,Monthly Subscriptions,11.48,0.52,12.0,0.29,0.23
Total,,,489.68,21.32,511.0,11.50,9.82



    ID: 2
    Number of Payments: 169
    Date of Payout: 13-Dec-2023
    Payout Amount: £2811.4
    


,Section,Payment Schedule,net_amount,total_fees,gross_amount,gocardless_fees,app_fees
0,Beavers,Activities,14.40,0.60,15.0,0.31,0.29
1,Beavers,Monthly Subscriptions,183.68,8.32,192.0,4.64,3.68
2,Scouts,Monthly Subscriptions,264.04,11.96,276.0,6.67,5.29
3,Squirrels,Monthly Subscriptions,68.88,3.12,72.0,1.74,1.38
Total,,,531.00,24.00,555.0,13.36,10.64
